In [11]:
import numpy as np
import pywt

import seaborn as sns #绘制confusion matrix heatmap

import sklearn
import os
import time

import warnings

warnings.simplefilter('ignore') #忽略警告

In [2]:
import scipy
import scipy.io as sio

import scipy.signal

from scipy import linalg

import pandas as pd
from sklearn.decomposition import PCA
#分类器
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

#from sklearn.lda import LDA
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split

#外源训练器
import xgboost
import lightgbm

#模型集成
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import StackingClassifier

#模型调节
from sklearn.model_selection import GridSearchCV #参数搜索
from mlxtend.feature_selection import SequentialFeatureSelector #特征选择函数 选择合适的feature

#结果可视化
from sklearn.metrics import classification_report , confusion_matrix #混淆矩阵

#相关指标
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

from sklearn.metrics import cohen_kappa_score

from sklearn.metrics import roc_auc_score

#二分类其多分类化
#from sklearn.multiclass import OneVsOneClassifier
#from sklearn.multiclass import OneVsRestClassifier

#from sklearn.preprocessing import StandardScaler
#from sklearn.cluster import KMeans

#距离函数 度量向量距离
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics.pairwise import cosine_similarity #余弦相似度

#one-hot使用
#from keras.utils import to_categorical

from sklearn.preprocessing import label_binarize

#绘图
import matplotlib.pyplot as plt

import scipy.linalg as la

import gc

%matplotlib inline

In [3]:
sample_rate = 256 #hz
origin_channel = 16 #5 channel eeg

#采集的通道
#共16 channel
#未使用的channel使用none代替
#reference:a study on performance increasing in ssvep based bci application
SAMPLE_CHANNEL = ['Pz' , 'PO3' , 'PO4' , 'O1' , 'O2' , 'Oz' , 'O9' , 'FP2' ,
                  'C4' , 'C6' , 'CP3' , 'CP1' ,
                  'CPZ' , 'CP2' , 'CP4' , 'PO8']


# 减去前多少秒数据 second
# 减去后多少秒数据 second
CLIP_FORWARD = 2
CLIP_BACKWARD = 1

# 单个小段的实验时长
trial_time = 3 #second

trial_offset = 0 #second
start_trial_time = 0 #真正的实验开始时刻
end_trial_time = 2 #真正的实验结束时刻(<trial_time)

#是否进行归一化
#reference:a study on performance increasing in ssvep based bci application
#IS_NORMALIZE = True

#是否进行滤波
#IS_FILTER = False
#EEG频率范围
#reference:a study on performance increasing in ssvep based bci application
LO_FREQ = 0.5
HI_FREQ = 40

#是否陷波
#IS_NOTCH = False
NOTCH_FREQ = 50 #陷波 工频



In [4]:
def load_data(filename):    
    data = sio.loadmat(file_name=filename)['data_received'] #length*16 matrix
    
    #截掉前后数据
    data = data[CLIP_FORWARD * sample_rate : - CLIP_BACKWARD * sample_rate]
    
    #滤波
    #return butter_worth(data , 0.5 , 30)

    return data

In [5]:
def separate(data , label , overlap_length = 128):
    '''
    最长重叠长度为size长 256*3 个数据点
    '''
    train_data = []
    train_labels = []

    size = sample_rate * trial_time #一小段 256*3 个数据点
    data_length = data.shape[0]

    idx = 0

    while idx<data_length-size:
        train_data.append(data[idx : idx+size , :])
        train_labels.append(label)

        idx = idx + (size - overlap_length)

    return np.array(train_data) , np.array(train_labels)

In [6]:
def shuffle_t_v(filenames):
    np.random.shuffle(filenames)
    
    return filenames

def combine(freq):
    overlap_length = 2*256 #重叠2秒数据
    
    #保证随机性 进行置乱
    person_0_filenames = shuffle_t_v( os.listdir('data/0/%s/' % freq) )
    person_1_filenames = shuffle_t_v( os.listdir('data/1/%s/' % freq) )
    person_2_filenames = shuffle_t_v( os.listdir('data/2/%s/' % freq) )
    person_3_filenames = shuffle_t_v( os.listdir('data/3/%s/' % freq) )
    person_4_filenames = shuffle_t_v( os.listdir('data/4/%s/' % freq) )
    person_5_filenames = shuffle_t_v( os.listdir('data/5/%s/' % freq) )
    person_6_filenames = shuffle_t_v( os.listdir('data/6/%s/' % freq) )
    person_7_filenames = shuffle_t_v( os.listdir('data/7/%s/' % freq) )
    person_8_filenames = shuffle_t_v( os.listdir('data/8/%s/' % freq) )

    #打开信号文件 并 合并
    person_0 = np.concatenate([load_data('data/0/%s/' % freq + filename) for filename in person_0_filenames] , axis = 0)
    person_1 = np.concatenate([load_data('data/1/%s/' % freq + filename) for filename in person_1_filenames] , axis = 0)
    person_2 = np.concatenate([load_data('data/2/%s/' % freq + filename) for filename in person_2_filenames] , axis = 0)
    person_3 = np.concatenate([load_data('data/3/%s/' % freq + filename) for filename in person_3_filenames] , axis = 0)
    person_4 = np.concatenate([load_data('data/4/%s/' % freq + filename) for filename in person_4_filenames] , axis = 0)
    person_5 = np.concatenate([load_data('data/5/%s/' % freq + filename) for filename in person_5_filenames] , axis = 0)
    person_6 = np.concatenate([load_data('data/6/%s/' % freq + filename) for filename in person_6_filenames] , axis = 0)
    person_7 = np.concatenate([load_data('data/7/%s/' % freq + filename) for filename in person_7_filenames] , axis = 0)
    person_8 = np.concatenate([load_data('data/8/%s/' % freq + filename) for filename in person_8_filenames] , axis = 0)
    
    #============
    #训练数据分段
    person_data_0 , person_labels_0 = separate(person_0 , label = 0 , overlap_length=overlap_length)
    person_data_1 , person_labels_1 = separate(person_1 , label = 1 , overlap_length=overlap_length)
    person_data_2 , person_labels_2 = separate(person_2 , label = 2 , overlap_length=overlap_length)
    person_data_3 , person_labels_3 = separate(person_3 , label = 3 , overlap_length=overlap_length)
    person_data_4 , person_labels_4 = separate(person_4 , label = 4 , overlap_length=overlap_length)
    person_data_5 , person_labels_5 = separate(person_5 , label = 5 , overlap_length=overlap_length)
    person_data_6 , person_labels_6 = separate(person_6 , label = 6 , overlap_length=overlap_length)
    person_data_7 , person_labels_7 = separate(person_7 , label = 7 , overlap_length=overlap_length)
    person_data_8 , person_labels_8 = separate(person_8 , label = 8 , overlap_length=overlap_length)

    #合并数据
    data = np.concatenate((person_data_0 , person_data_1 , person_data_2 ,
                           person_data_3 , person_data_4 , person_data_5 ,
                           person_data_6 , person_data_7 , person_data_8 
                        ))
    
    labels = np.concatenate((person_labels_0 , person_labels_1 , person_labels_2 ,
                             person_labels_3 , person_labels_4 , person_labels_5 ,
                             person_labels_6 , person_labels_7 , person_labels_8 
                        ))
    
    return data , labels

In [7]:
def butter_worth(data , lowcut , highcut , order=6):
    nyq = 0.5 * sample_rate
    
    lo = lowcut / nyq
    hi = highcut / nyq
    
    b,a = scipy.signal.butter(order , [lo , hi] , btype='bandpass')

    return np.array([scipy.signal.filtfilt(b , a , data[: , i]) for i in range(data.shape[1])]).reshape((-1 , origin_channel))

In [8]:
def alpha_subBP_features(data):
    alpha1 = butter_worth(data , 8.5 , 11.5)
    alpha2 = butter_worth(data , 9.0 , 12.5)    
    alpha3 = butter_worth(data , 9.5 , 13.5)   #11.5 后
    alpha4 = butter_worth(data , 8.0 , 10.5)   
    
    return np.array([alpha1 , alpha2 , alpha3 , alpha4])

def beta_subBP_features(data):
    beta1 = butter_worth(data , 15.0 , 30.0) #14.0 前
    beta2 = butter_worth(data , 16.0 , 17.0)    
    beta3 = butter_worth(data , 17.0 , 18.0)    
    beta4 = butter_worth(data , 18.0 , 19.0)    
    
    return np.array([beta1 , beta2 , beta3 , beta4])

def powermean(data):
    #官方demo跳4秒 前4秒为准备阶段
    return  np.power(data[ : , 0] , 2).mean(), \
            np.power(data[ : , 1] , 2).mean(), \
            np.power(data[ : , 2] , 2).mean(), \
            np.power(data[ : , 3] , 2).mean(), \
            np.power(data[ : , 4] , 2).mean(), \
            np.power(data[ : , 5] , 2).mean(), \
            np.power(data[ : , 6] , 2).mean(), \
            np.power(data[ : , 7] , 2).mean(), \
            np.power(data[ : , 8] , 2).mean(), \
            np.power(data[ : , 9] , 2).mean(), \
            np.power(data[ : , 10] , 2).mean(), \
            np.power(data[ : , 11] , 2).mean(), \
            np.power(data[ : , 12] , 2).mean(), \
            np.power(data[ : , 13] , 2).mean(), \
            np.power(data[ : , 14] , 2).mean(), \
            np.power(data[ : , 15] , 2).mean()       
            
def log_subBP_feature_extraction(alpha , beta):
    #alpha
    power_1_a = powermean(alpha[0])
    power_2_a = powermean(alpha[1])
    power_3_a = powermean(alpha[2])
    power_4_a = powermean(alpha[3])
    
    #beta
    power_1_b = powermean(beta[0])
    power_2_b = powermean(beta[1])
    power_3_b = powermean(beta[2])
    power_4_b = powermean(beta[3])
    
    X= np.array(
        [np.log(power_1_a) ,
         np.log(power_2_a) ,
         np.log(power_3_a) ,
         np.log(power_4_a) ,
         np.log(power_1_b) ,
         np.log(power_2_b) ,
         np.log(power_3_b) ,
         np.log(power_4_b)
        ]
        ).flatten()

    return X

def feature_extraction_sub_band_power(data):
    n_features = 128
    X = np.zeros((data.shape[0] , n_features))
    
    for i , datum in enumerate(data):
        #对于每一个样本 小段（3秒小段）
        alpha = alpha_subBP_features(datum)
        beta = beta_subBP_features(datum)
            
        X[i, :] = log_subBP_feature_extraction(alpha , beta)

    return X

# stacking  
# svm knn 'naive bayes' 'decision tree'  

In [9]:
acc_s = []

estimators = [
    ('svm', SVC()),
    ('knn', KNeighborsClassifier()),
    ('bayes' , GaussianNB()),
    ('decision' , DecisionTreeClassifier()),
            ]

for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data_feature = feature_extraction_sub_band_power(data) #特征提取
    
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = StackingClassifier(estimators=estimators , final_estimator = RandomForestClassifier(n_estimators=20))
        X_train, X_test, y_train, y_test = train_test_split(data_feature , labels , test_size = 0.1)
        
        classifier.fit(X_train , y_train)

        acc = classifier.score(X_test , y_test)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  6
0.6763925729442971
0.6604774535809018
0.6896551724137931
0.6936339522546419
0.7082228116710876
0.6949602122015915
0.7015915119363395
0.6724137931034483
0.6724137931034483
0.7042440318302388
0.6750663129973474
0.6843501326259946
0.6538461538461539
0.6591511936339522
0.7015915119363395
0.7015915119363395
0.6684350132625995
0.6578249336870027
0.6604774535809018
0.7068965517241379
freq =  6 average_acc =  0.6821618037135277 var_acc =  0.00032967849629561894
freq =  7.5
0.6777188328912467
0.6604774535809018
0.7095490716180372
0.6684350132625995
0.6896551724137931
0.6737400530503979
0.6803713527851459
0.6803713527851459
0.6989389920424404
0.6883289124668435
0.6896551724137931
0.6803713527851459
0.6962864721485411
0.6631299734748011
0.6870026525198939
0.6976127320954907
0.6445623342175066
0.7175066312997348
0.7148541114058355
0.7015915119363395
freq =  7.5 average_acc =  0.6860079575596817 var_acc =  0.00032890455149899055
freq =  8.5
0.7082228116710876
0.7108753315649867
0.72679045

In [ ]:
acc_s = []

estimators = [
    ('svm', SVC()),
    ('knn', KNeighborsClassifier()),
    ('bayes' , GaussianNB()),
    ('decision' , DecisionTreeClassifier()),
            ]

for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data_feature = feature_extraction_sub_band_power(data) #特征提取
    
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = StackingClassifier(estimators=estimators , final_estimator = RandomForestClassifier(n_estimators=20))
        X_train, X_test, y_train, y_test = train_test_split(data_feature , labels , test_size = 0.1)
        
        classifier.fit(X_train , y_train)

        acc = classifier.score(X_test , y_test)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

In [ ]:
acc_s = []

estimators = [
    ('svm', SVC()),
    ('knn', KNeighborsClassifier()),
    ('bayes' , GaussianNB()),
    ('decision' , DecisionTreeClassifier()),
            ]

for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq = ' , freq)
    
    data , labels = combine(freq) #读取数据
    data_feature = feature_extraction_sub_band_power(data) #特征提取
    
    
    classifier = StackingClassifier(estimators=estimators , final_estimator = RandomForestClassifier(n_estimators=20))
    X_train, X_test, y_train, y_test = train_test_split(data_feature , labels , test_size = 0.1)

    start = time.clock()
    
    classifier.fit(X_train , y_train)
    acc = classifier.score(X_test , y_test)
    
    all_time = time.clock() - start
    
    print(all_time)

freq =  6
